# Table 2 Recreation

Table 2 in the paper is used to show the tendency of models to rely on background signal. This notebook shows a recreation of the left hand side of the table which analyses models that have been pre-trained on ImageNet. Table2RHS.ipynb shows the recreation of the right hand side which analyses models that are trained on IN-9L, a 9 class alternative to ImageNet.

Summary: 

Test set | MobileNetV3-Large | EfficientNet | ResNet-50 | DPN-92 | WRN50x2
--- | --- | --- | --- | --- | ---
original | 95.0% | 94.7% | 95.2% | 97.2% | 96.6%
only_bg_t  | 16.00% | 10.6% | 16.8% | 17.6% | 18.8%
mixed_same  | 81.7% | 87.1%| 80.5% | 90.5% | 88.3%
mixed_rand  | 70.3% | 80.2% | 69.8% | 86.0% | 81.4%
BG-Gap | 11.4% | 6.9% | 10.7% | 4.5% | 6.9%

The main point that the paper is emphasising still holds event though there is slight discrepencies in most accuracy figures. Clearly these models also somewhat rely on the background signal.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip drive/MyDrive/ReproducabilityChallenge/only_bg_t.zip
!unzip drive/MyDrive/ReproducabilityChallenge/mixed_rand.zip
!unzip drive/MyDrive/ReproducabilityChallenge/original.zip
!unzip drive/MyDrive/ReproducabilityChallenge/mixed_same.zip

Streaming output truncated to the last 5000 lines.
  inflating: mixed_same/val/04_carnivore/fg_n02115641_41341_bg_n02233338_15470.JPEG  
  inflating: __MACOSX/mixed_same/val/04_carnivore/._fg_n02115641_41341_bg_n02233338_15470.JPEG  
  inflating: mixed_same/val/04_carnivore/fg_n02128757_04135_bg_n02279972_20458.JPEG  
  inflating: __MACOSX/mixed_same/val/04_carnivore/._fg_n02128757_04135_bg_n02279972_20458.JPEG  
  inflating: mixed_same/val/04_carnivore/fg_n02128757_22170_bg_n02264363_47939.JPEG  
  inflating: __MACOSX/mixed_same/val/04_carnivore/._fg_n02128757_22170_bg_n02264363_47939.JPEG  
  inflating: mixed_same/val/04_carnivore/fg_n02119022_00157_bg_n02177972_01737.JPEG  
  inflating: __MACOSX/mixed_same/val/04_carnivore/._fg_n02119022_00157_bg_n02177972_01737.JPEG  
  inflating: mixed_same/val/04_carnivore/fg_n02119022_06668_bg_n02276258_45550.JPEG  
  inflating: __MACOSX/mixed_same/val/04_carnivore/._fg_n02119022_06668_bg_n02276258_45550.JPEG  
  inflating: mixed_same/val/04_car

In [ ]:
# Execute this code block to install dependencies when running on colab
try:
    import torch
except:
    from os.path import exists
    from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
    platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
    cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
    accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

    !pip install -q http://download.pytorch.org/whl/{accelerator}/torch-1.0.0-{platform}-linux_x86_64.whl torchvision

try: 
    import torchbearer
except:
    !pip install torchbearer
    import torchbearer

     |████████████████████████████████| 143kB 7.7MB/s 


In [ ]:
# automatically reload external modules if they change
%load_ext autoreload
%autoreload 2

import torch.nn.functional as F
import torchvision.transforms as transforms
from torch import nn
from torch import optim
from torch.utils.data import DataLoader
from torchbearer import Trial
from torchvision.datasets import ImageFolder
import os
import json
import torchvision
import numpy as np

In [ ]:
#Convert raw data to DataLoader object
def getDataLoader(datadir = 'mixed_same/val'):
    # convert each image to tensor format
    preprocess = transforms.Compose([transforms.ToTensor(), 
                                    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

    #datadir = '/Volumes/DRIVE/test_sets/mixed_same/val'
    #datadir = os.path.join('/Volumes/DRIVE/test_sets/mixed_same/', 'val')
    imgFolder = ImageFolder(datadir, transform=preprocess)
    testloader = DataLoader(imgFolder, batch_size=32, shuffle=False)
    return testloader

In [ ]:
#Function to compute model accuracy
def getModelAccuracyCuda(model, testloader, in_to_in9):
    # Compute the model accuracy on the test set
    correct = 0
    total = 0

    for inputs, labels in testloader:
        outputs = model(inputs.to('cuda:0'))

        for (output, real_label) in zip(outputs, labels):
            model_output = in_to_in9[str(torch.argmax(output).item())]

            if model_output == real_label.item():
                correct += 1
            total += 1


    print('Test Accuracy: %2.2f %%' % ((100.0 * correct) / total))


In [ ]:
#Function retrieves the imagenet to in9l class converter dictionary
def getIn2In9(root = ''):
    with open(root + 'in_to_in9.json') as f:
      in_to_in9 = json.load(f)
    return in_to_in9

In [ ]:
#Get the imagenet to in9l class converter
in_to_in9 = getIn2In9(root='drive/MyDrive/ReproducabilityChallenge/')

In [ ]:
#Get the mixed_same dataset
mixed_same = getDataLoader(datadir = 'mixed_same/val')

In [ ]:
#Get the mixed_rand dataset
mixed_rand = getDataLoader(datadir = 'mixed_rand/val')

In [ ]:
#Get the original dataset
original = getDataLoader(datadir = 'original/val')

In [ ]:
#Get the only_bg_t dataset
only_bg_t = getDataLoader(datadir = 'only_bg_t/val')

# MobileNetV3-Large: Pytorch Implementation

In [ ]:
mv3 = torchvision.models.mobilenet_v3_large(pretrained=True)
mv3.to('cuda:0')
mv3.eval()

Downloading: "https://download.pytorch.org/models/mobilenet_v3_large-8738ca79.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v3_large-8738ca79.pth


MobileNetV3(
  (features): Sequential(
    (0): ConvBNActivation(
      (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): Hardswish()
    )
    (1): InvertedResidual(
      (block): Sequential(
        (0): ConvBNActivation(
          (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (1): ConvBNActivation(
          (0): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): Identity()
        )
      )
    )
    (2): InvertedResidual(
      (block): Sequential(
        (0): ConvBNActivation(
          (0): Conv2d(16, 64, kernel_size=(1, 1), stride=

In [ ]:
#Compute model accuracy on mixed_same
getModelAccuracyCuda(mv3, mixed_same, in_to_in9)

Test Accuracy: 81.68 %


In [ ]:
#Compute model accuracy on mixed_rand
getModelAccuracyCuda(mv3, mixed_rand, in_to_in9)

Test Accuracy: 70.25 %


In [ ]:
#Compute model accuracy on original
getModelAccuracyCuda(mv3, original, in_to_in9)

Test Accuracy: 95.01 %


In [ ]:
#Compute model accuracy on only_bg_t
getModelAccuracyCuda(mv3, only_bg_t, in_to_in9)

Test Accuracy: 16.00 %


# EfficientNet-b0
Model provided by https://github.com/lukemelas/EfficientNet-PyTorch#usage

In [ ]:
!pip install efficientnet_pytorch

  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.0-cp37-none-any.whl size=16031 sha256=e3da2f956b071aba13d750333d742fa0b95942224bbfefd2b057406803edb9f1
  Stored in directory: /root/.cache/pip/wheels/e9/c6/e1/7a808b26406239712cfce4b5ceeb67d9513ae32aa4b31445c6
Successfully built efficientnet-pytorch


In [ ]:
from efficientnet_pytorch import EfficientNet
efficientnet = EfficientNet.from_pretrained('efficientnet-b0')
efficientnet.to('cuda:0')
efficientnet.eval()

Loaded pretrained weights for efficientnet-b0


EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
  )
  (_bn0): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        32, 32, kernel_size=(3, 3), stride=[1, 1], groups=32, bias=False
        (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
      )
      (_bn1): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
      (_se_reduce): Conv2dStaticSamePadding(
        32, 8, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        8, 32, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        32, 16, kernel_size=

In [ ]:
getModelAccuracyCuda(efficientnet, mixed_same, in_to_in9)

Test Accuracy: 87.11 %


In [ ]:
#Compute model accuracy on mixed_rand
getModelAccuracyCuda(efficientnet, mixed_rand, in_to_in9)

Test Accuracy: 80.17 %


In [ ]:
#Compute model accuracy on original
getModelAccuracyCuda(efficientnet, original, in_to_in9)

Test Accuracy: 94.74 %


In [ ]:
#Compute model accuracy on only_bg_t
getModelAccuracyCuda(efficientnet, only_bg_t, in_to_in9)

Test Accuracy: 10.57 %


# DPN-92
Model provided by https://github.com/rwightman/pytorch-dpn-pretrained

In [ ]:
dpn = torch.hub.load('rwightman/pytorch-dpn-pretrained', 'dpn92', pretrained=True)
dpn.to('cuda:0')
dpn.eval()

Using cache found in /root/.cache/torch/hub/rwightman_pytorch-dpn-pretrained_master


DPN(
  (features): Sequential(
    (conv1_1): InputBlock(
      (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      (act): ReLU(inplace=True)
      (pool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    )
    (conv2_1): DualPathBlock(
      (c1x1_w_s1): BnActConv2d(
        (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
        (act): ReLU(inplace=True)
        (conv): Conv2d(64, 288, kernel_size=(1, 1), stride=(1, 1), bias=False)
      )
      (c1x1_a): BnActConv2d(
        (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
        (act): ReLU(inplace=True)
        (conv): Conv2d(64, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
      )
      (c3x3_b): BnActConv2d(
        (bn): BatchNorm2d(96, eps=0.001, momentum=0.1, affine=True, track_running_s

In [ ]:
getModelAccuracyCuda(dpn, mixed_same, in_to_in9)

Test Accuracy: 90.47 %


In [ ]:
getModelAccuracyCuda(dpn, mixed_rand, in_to_in9)

Test Accuracy: 85.98 %


In [ ]:
getModelAccuracyCuda(dpn, original, in_to_in9)

Test Accuracy: 97.21 %


In [ ]:
getModelAccuracyCuda(dpn, only_bg_t, in_to_in9)

Test Accuracy: 17.58 %


# WRN 50x2

In [ ]:
import torchvision.models as models
wrn = models.wide_resnet50_2(pretrained=True)
wrn.to('cuda:0')
wrn.eval()

Downloading: "https://download.pytorch.org/models/wide_resnet50_2-95faca4d.pth" to /root/.cache/torch/hub/checkpoints/wide_resnet50_2-95faca4d.pth


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), strid

In [ ]:
getModelAccuracyCuda(wrn, mixed_same, in_to_in9)

Test Accuracy: 88.27 %


In [ ]:
getModelAccuracyCuda(wrn, mixed_rand, in_to_in9)

Test Accuracy: 81.36 %


In [ ]:
getModelAccuracyCuda(wrn, original, in_to_in9)

Test Accuracy: 96.64 %


In [ ]:
getModelAccuracyCuda(wrn, only_bg_t, in_to_in9)

Test Accuracy: 18.81 %
